In [204]:
from dash import Dash, dash_table, dcc, callback, Output, Input, clientside_callback, _dash_renderer
import pandas as pd
import plotly.express as px
import dash_mantine_components as dmc
from requests import get
from dash.dependencies import MATCH, ALL, State
from dash.exceptions import PreventUpdate
from dash import callback_context
from dash import html
import webbrowser
from threading import Timer
from dash_iconify import DashIconify


In [205]:
CSV_FILES = {
    "Category": "C:/Users/kaila/OneDrive/Desktop/COMP3610PROJECT/notebooks/dashboard/CPS_Summary.csv",
    "Brands per Category": "C:/Users/kaila/OneDrive/Desktop/COMP3610PROJECT/notebooks/dashboard/BPCPS_Summary.csv",
    "Products per Brand": "C:/Users/kaila/OneDrive/Desktop/COMP3610PROJECT/notebooks/dashboard/PPBPS_Summary.csv",
    "Products per Category": "C:/Users/kaila/OneDrive/Desktop/COMP3610PROJECT/notebooks/dashboard/PPCPS_Summary.csv"
}


In [206]:
type_map = {
        "Brands per Category": "brand",
        "Products per Category": "product",
        "Products per Brand": "product",
        "Category": "category"
    }

In [207]:
def load_data(radio_choice):
    col = type_map[radio_choice]
    df = pd.read_csv(
        CSV_FILES[radio_choice],
        usecols=[col, 'year', 'month','Popularity Score']
    )
    return df

In [208]:
def plot_time_series(filtered_df, radio_choice, selected_val, template):
    fig = px.line(filtered_df, x="date", y="Popularity Score",
                  template=template, title=f"{radio_choice} - {selected_val}")
    
    return fig

In [209]:
def plot_bar_chart(filtered_df, prod_names, selected_val, template):
    fig = px.bar(filtered_df, x="month_year", y='Popularity Score',
                 template=template, title=f"Top Products in {selected_val}:\n{', '.join(prod_names.astype(str))}")
    
    return fig

In [210]:
def get_template(theme):
    return "plotly_dark" if theme == "dark" else "plotly_white"

In [211]:
theme_toggle = dmc.Switch(
    offLabel=DashIconify(icon="radix-icons:sun", width=15, color=dmc.DEFAULT_THEME["colors"]["yellow"][8]),
    onLabel=DashIconify(icon="radix-icons:moon", width=15, color=dmc.DEFAULT_THEME["colors"]["yellow"][6]),
    id="color-scheme-switch",
    persistence=True,
    color="grey",
)

In [212]:
def load_brand_data(radio_choice, select_cat, select_brand, template):
    # df = pd.read_csv(
    #     CSV_FILES[radio_choice],
    #     usecols=['brand', 'year', 'month', 'Popularity Score']
    # )
    
    if not select_cat:
        return(
            [],
            "Please select a category to begin viewing performance",
            {'display': 'block'}, [],
            {'display': 'block'}
        )

    cat_df = load_data("Category")
    brand_df = pd.read_csv(CSV_FILES[radio_choice])

    cat_options = [{'label': val, 'value': val} for val in cat_df['category'].unique()]

    brand_df = brand_df[brand_df['category'] == select_cat].copy()

    brand_options = [{'label': val, 'value': val} for val in brand_df['brand'].unique()]

    if select_brand not in [opt["value"] for opt in brand_options]:
        return(
            cat_options,
            "Please select a category to begin viewing performance",
            {'display': 'block'}, [],
            {'display': 'block'}
        )
    
    filtered_df = brand_df[brand_df['brand'] == select_brand].copy()
    filtered_df['date'] = pd.to_datetime(filtered_df[["year", "month"]].assign(day=1))

    
    return(
        cat_options,
        dcc.Graph(figure=plot_time_series(filtered_df, radio_choice, select_brand, template)),
        {'display': 'block'}, brand_options,
        {'display': 'block'},
    )

# Output("first-dropdown", "data"),
#     Output("plots", "children"),
#     Output("compare-checkbox", "style"),
#     Output("second-dropdown", "data"),
#     Output('second-dropdown', 'style'),

In [213]:
def load_prod_data(radio_choice, selected_val, template):
    # df = pd.read_csv(
    #     CSV_FILES[radio_choice],
    #     usecols=['product', 'year', 'month', 'Popularity Score']
    # )

    df = load_data(radio_choice)

    filtered_df = df.head(6)
    # print(filtered_df.head())
    options = [{'label': str(y), 'value': str(y)} for y in sorted(filtered_df['year'].unique())]

    if selected_val not in [opt["value"] for opt in options]:
        return options, "Please select a product to view the populartiy  data.", {'display': 'none'}
    
    filtered_df = filtered_df[filtered_df['year'].astype(str) == selected_val].copy()
    prod_names = filtered_df['product']
    # print(prod_names)

    filtered_df['month_year'] = filtered_df['month'].astype(str) + "-" + filtered_df['year'].astype(str)

    return(
        options, 
        dcc.Graph(figure=plot_bar_chart(filtered_df, prod_names, selected_val, template)), 
        {'display': 'none'}, [],
        {'display': 'none'},
    )

In [214]:
def load_cat_data(radio_choice, selected_val, template, select_brand):
    template = get_template(template)

    df = load_data(radio_choice)

    options = [{'label': val, 'value': val} for val in sorted(df['category'].unique())]

    if selected_val not in [opt["value"] for opt in options]:
        return(
            options, "Please select a category to view the time series data.", 
            {'display': 'block'}, select_brand, {'display': 'none'}
        )
    
    filtered_df = df[df['category'] == selected_val].copy()
    filtered_df['date'] = pd.to_datetime(filtered_df[["year", "month"]].assign(day=1))

    return (
        options, 
        dcc.Graph(figure=plot_time_series(filtered_df, radio_choice, selected_val, template)), 
        {'display': 'block'}, 
        select_brand,
        {'display': 'none'}
    )

In [215]:
app = Dash()

app.layout = dmc.MantineProvider(
    children=[
        theme_toggle,
        dcc.Store(id='theme-store'),
        dmc.Container([

            dmc.Title("Market Analysis Dashboard", 
                      style={"size":"h3", "textAlign": "center"}), 
        
            dmc.Grid([
                dmc.GridCol(
                    dmc.RadioGroup(
                    [dmc.Radio(i, value=i) for i in CSV_FILES.keys()],
                    id="radio-group",
                    value="Category",
                    size="md",
                    style={"marginBottom": "25px"}
                    ),
                    span=8
                ),

                dmc.GridCol(
                    dmc.Checkbox(
                    id="compare-checkbox",
                    labelPosition="right",
                    label="Compare",
                    color="#5c7cfa",
                    variant="filled",
                    size="sm",
                    radius="sm",
                    disabled=False,
                    indeterminate=False,
                    style={'display': 'block', 'justifyContent': 'center',
                        'margonBottom': '25px'},
                    ),
                    span=4,
                    style={"display": "flex", 
                           "alignItems": "top", 
                           "justifyContent": "flex-end"}
                ),
            ]),
            
            dmc.Select(
                id="first-dropdown",
                data=[],
                placeholder="Select",
                searchable=True,
                clearable=True,
                nothingFoundMessage="No matches found",
            ),

            dmc.Space(h=20),

            dmc.Select(
                id='second-dropdown',
                data=[],
                placeholder="Select",
                searchable=True,
                clearable=True,
                nothingFoundMessage="No matches found",
                style={'display': 'none'},
            ),

            dmc.Space(h=20),

            html.Div(id="plots")
        ], fluid=True,
    )],
)
@callback(
    Output("first-dropdown", "data"),
    Output("plots", "children"),
    Output("compare-checkbox", "style"),
    Output("second-dropdown", "data"),
    Output('second-dropdown', 'style'),
    Input("radio-group", "value"),
    Input("first-dropdown", "value"),
    Input("theme-store", "data"),
    Input('second-dropdown', 'value'),
)
def combined_update(radio_choice, select_cat, select_brand, theme):
    template = get_template(theme)

    col_map = {
        "Brands per Category": load_brand_data,
        "Products per Category": load_prod_data,
        "Products per Brand": load_prod_data,
        "Category": load_cat_data
    }

    if radio_choice in col_map:
        return col_map[radio_choice](radio_choice, select_cat, select_brand, template)
    
    return [], "Invalid selection.", {'display': 'none'}, []
    
clientside_callback(
    """
    (switchOn) => {
       const theme = switchOn ? 'dark' : 'light';
        document.documentElement.setAttribute('data-mantine-color-scheme', theme);
        return theme;
    }
    """,
    Output("theme-store", "data"),
    Input("color-scheme-switch", "checked"),
)


# def open_browser():
#     webbrowser.open_new("http://127.0.0.1:8050/")

if __name__ == "__main__":
    # Timer(1, open_browser).start()  # Wait 1 second then open browser
    app.run(debug=True)